<a href="https://colab.research.google.com/github/Yuji-ONUKI/GCI2020_Winter/blob/main/Tokyo%EF%BC%BFStan_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ここから、Rスクリプトを実行してみます。


In [ ]:
!apt-get install -y fonts-noto-cjk

In [ ]:
library(ggplot2)
par(family = "NotoSansCJKkr-Bold")
theme(base_family = "NotoSansCJKkr-Bold")


In [ ]:
install.packages('rstan')

In [ ]:
library(tidyverse)
library(lubridate)

In [9]:
##########################################################################
#独立な2要因の実験データを分析
#■入力
#y: ベクトル形式の特性値
#A: ベクトル形式の水準(1からaまで、抜けなしで、整数で指定)
#B: ベクトル形式の水準(1からbまで、抜けなしで、整数で指定)
#prior=F : 論理値。Tなら事前分布の範囲を指定。Fなら指定せず。
#mL=-1000, mH=1000, sL=0, sH=100 : 事前分布のパラメタ
#prob=c(0.025, 0.05, 0.5, 0.95, 0.975): 事後分布で報告する確率点
#see=1234,cha=5,war=1000,ite=21000 : MCMC関連のパラメタ
#■出力
# fit:stanの出力, par:母数リスト, prob:確率ベクトル, 
#mu;全平均, [a]muA;A平均, [b]muB;B平均, [a,b]muAB;交互作用, 
#[a,b]cellmean;セル平均値, sigmaA;A標準偏差, sigmaB;B標準偏差,sigmaAB;AB標準偏差
#sigmaE;E標準偏差, eta2A;A説明率, eta2B;B説明率, eta2AB;AB説明率, eta2T;全説明率
#deltaA;A効果量, deltaB;B効果量, deltaAB;AB効果量, UbigA[a];A論理値0以上確率, 
#UsmaA[a];A論理値0以下確率, UbigB[b];B論理値0以上確率, UsmaB[b];B論理値0以下確率
#UbigAB[a,b];AB論理値0以上確率, UsmaAB[a,b];AB論理値0以下確率, 
#U2A[a,a];Aの2水準比較, U2B[b,b];Bの2水準比較, log_lik;対数尤度
##########################################################################
E2Ind <- function(y,B,prior=F,mL=-1000, mH=1000, sL=0, sH=100,prob=c(0.025, 0.05, 0.5, 0.95, 0.975),see=1234, cha=5, war=1000, ite=2000, fi=NA)
{
   library(rstan)
   if (prior) {
     dat <- list(n=length(y),b=length(unique(B)),
                 y=y,B=B,mL=mL,mH=mH,sL=sL,sH=sH)
     scr <- "E2IndPT_c.stan"                           # Stan ファイル名
   } else {
     dat <- list(n=length(y),b=length(unique(B)),
                 y=y,B=B)
     scr <- "E2IndPF_c.stan"                           # Stan ファイル名
   }
   par<-c("mu","muB","cellmean","sigmaB",
       "sigmaE","eta2T","deltaB",
       "log_lik")
   fit<-stan(file=scr,data=dat,pars=par,seed=see,chains=cha,warmup=war,iter=ite, fit=fi)
   ext<-extract(fit,par);
   out<-list(prior=prior,fit=fit,par=par,prob=prob,
      mu=ext$mu,muB=ext$muB,cellmean=ext$cellmean,
      sigmaB=ext$sigmaB,sigmaE=ext$sigmaE,
      deltaB=ext$deltaB,
      log_lik=ext$log_lik)
   class(out)<-'E2Ind'
   return(invisible(out))
}
##########################################################################
#印刷のメソッド
#■入力
#x:クラス 'E2Ind'のオブジェクト
#degits=3 : 小数の丸め
##########################################################################
print.E2Ind<-function(x,degits=3)
{
   prior<-x$prior;log_lik<-x$log_lik;
   if (prior) {print("******************事前分布指定******************")}
         else {print("******************事前分布デフォルト************")}
   waic<- (-2)*(log(mean(exp(log_lik)))) + 2*(var(log_lik))
   print(x$fit,pars=x$par,digits_summary=degits,probs=x$prob)
   print(paste("waic=",round(waic,degits),sep=""))
   out<-list(waic=waic)
   return(invisible(out))
}  
##########################################################################
#２つのセルの比較
#■入力
#x:クラス 'E2Ind'のオブジェクト
#degits=3 : 小数の丸め
#H="A": 固定する因子　A or B
#F=1: 整数、固定する因子の水準
#I=1: 整数、比較する因子の平均値の大きい水準
#J=2: 整数、比較する因子の平均値の小さい
#cr1: 閾上率の基準値
##########################################################################
E2betw_level<-function(x,degits=3,H="A",F=1,I=1,J=2,cr1=F)
{
   if (H=='A') { big<-x$cellmean[,F,I];sma<-x$cellmean[,F,J];比較<-"B";}
          else { big<-x$cellmean[,I,F];sma<-x$cellmean[,J,F];比較<-"A";}
   G<-matrix(0,length(x$mu),5)
   G[,1] <- big-sma;
   G[,2] <- G[,1]/x$sigmaE;
   G[,3] <- pnorm(big,sma,x$sigmaE);
   G[,4] <- pnorm((G[,2]/sqrt(2)), 0.0, 1.0);
   lab<-c("平均値の差","効果量","非重複度","優越率")
   co<-4
   if(is.numeric(cr1)){  co<-co+1;
      G[,5] <- pnorm(((G[,1]-cr1)/(sqrt(2)*x$sigmaE)), 0.0, 1.0);
      lab<-c(lab,paste("閾上率(",cr1,")",sep=""));}
   Gc<-cbind(
      apply(G[,1:co],2,mean),
      apply(G[,1:co],2,sd),
      t(apply(G[,1:co],2,quantile, probs=c(0.025, 0.05, 0.5, 0.95, 0.975)))
   )
   colnames(Gc)<-c("EAP","post.sd","2.5%","5%","50%","95%","97.5%")
   rownames(Gc)<-lab
   cat("要因",H,"の水準",F,"を固定して、要因",比較,"の水準",I,J,"を比較する¥n")
   round(Gc,degits)
}


In [23]:
##########################################################################
#独立な2群の差を推測する
#■入力
#x1:ベクトル形式の第1群のデータ (平均値の大きな群)
#x2:ベクトル形式の第2群のデータ (平均値の小さな群)
#EQU=1 : 論理値。1なら等分散。0なら異なる分散。
#prior=F : 論理値。Tなら事前分布の範囲を指定。Fなら指定せず。
#mL=-1000, mH=1000, sL=0, sH=100 : 事前分布のパラメタ
#prob=c(0.025, 0.05, 0.5, 0.95, 0.975): 事後分布で報告する確率点
#see=1234,cha=5,war=1000,ite=21000 : MCMC関連のパラメタ
#■出力
# fit:stanの出力, par:母数リスト, prob:確率ベクトル, 
# mu1:第1群平均, mu2:第2群平均, sigma1:第1群標準偏差, sigma2:第2群標準偏差, 
# xaste1:予測分布1, xaste2:予測分布2, log_lik:対数尤度
##########################################################################
G2Ind <- function(x1,x2,EQU=1,prior=F,mL=-1000, mH=1000, sL=0, sH=100,
         prob=c(0.025, 0.05, 0.5, 0.95, 0.975),
         see=1234, cha=5, war=1000, ite=21000, fi=NA)
{
   library(rstan)
   if (prior) {
     dat <- list(n1=length(x1),n2=length(x2),x1=x1,x2=x2,EQU=EQU,prior=prior,mL=mL,mH=mH,sL=sL,sH=sH)
     scr <- "G2IndPT.stan"                           # Stan ファイル名
   } else {
     dat <- list(n1=length(x1),n2=length(x2),x1=x1,x2=x2,EQU=EQU)
     scr <- "G2IndPF.stan"                           # Stan ファイル名
   }
   par<-c("mu","sigma1","sigma2","xaste","log_lik")# サンプリング変数
   fit<-stan(file=scr,data=dat,pars=par,seed=see,
             chains=cha,warmup=war,iter=ite, fit=fi)
   ext<-extract(fit, par)
   mu1<-ext$mu[,1]; mu2<-ext$mu[,2]; sigma1<-ext$sigma1; sigma2<-ext$sigma2; 
   xaste1<-ext$xaste[,1]; xaste2<-ext$xaste[,2]; log_lik<-ext$log_lik
   out<-list(EQU=EQU,prior=prior,fit=fit,par=par,prob=prob,mu1=mu1,mu2=mu2,sigma1=sigma1,sigma2=sigma2,xaste1=xaste1,xaste2=xaste2,log_lik=log_lik)
   class(out)<-'G2Ind'
   return(invisible(out))
}
##########################################################################
#印刷のメソッド
#■入力
#x:クラス 'G2Ind'のオブジェクト
#degits=3 : 小数の丸め
#cr1=F : 平均値の差の基準点
#cr2=F : 閾上率の基準点
#cr3=F : 効果量の基準点
#pr1=F : 非重複度の基準確率
#pr2=F : 優越率の基準確率
#pr3=F : 閾上率の基準確率
##########################################################################
print.G2Ind<-function(x,degits=3,cr1=F,cr2=F,cr3=F,pr1=F,pr2=F,pr3=F)
{
   EQU<-x$EQU; prior<-x$prior;
   if (EQU>0.5) {print("******************等分散モデル******************")}
           else {print("******************異分散モデル******************")}
   if (prior>0.5) {print("******************事前分布指定******************")}
             else {print("******************事前分布デフォルト******************")}
   mu1<-x$mu1; mu2<-x$mu2; sigma1<-x$sigma1; sigma2<-x$sigma2; 
   xaste1<-x$xaste1; xaste2<-x$xaste2; log_lik<-x$log_lik
   prob<-x$prob
   G<-matrix(0,length(mu1),7)
   G0<-sqrt(sigma1^2 + sigma2^2)
   G[,1] <- mu1 - mu2;
   G[,2] <- G[,1]/sigma1;
   G[,3] <- G[,1]/sigma2;
   G[,4] <- pnorm(mu1,mu2,sigma2); #U3
   G[,5] <- 1-pnorm(mu2,mu1,sigma1); #U3
   G[,6] <- pnorm((G[,1]/G0), 0.0, 1.0); #π_d
   co<-6; lab<-c("平均値の差","効果量1","効果量2","非重複度1群","非重複度2群","優越率")
   if(is.numeric(cr2)){co<- co+1;G[,co]<- pnorm(((G[,1]-cr2)/G0), 0.0, 1.0);
      lab<-c(lab,paste("閾上率(",cr2,")",sep=""))}
   Gc<-cbind(
      apply(G[,1:co],2,mean),
      apply(G[,1:co],2,sd),
      t(apply(G[,1:co],2,quantile, probs=prob))
   )
   colnames(Gc)<-c("EAP","post.sd",prob)
   rownames(Gc)<-lab
   U<-matrix(0,length(mu1),11)
   U[,1] <- ifelse(G[,1]>0.0,  1, 0);
   U[,2] <- ifelse(G[,1]<=0.0, 1, 0);
   U[,3] <- ifelse(xaste1-xaste2>0.0, 1, 0);
   co<-3;lab<-c("μ1-μ2が0より大きい確率","μ1-μ2が0以下の確率","優越率(直接比較)")
   if(is.numeric(cr2)){co<- co+1;U[,co]<- ifelse(xaste1-xaste2> cr2,1, 0);
         lab<-c(lab,paste("閾上率(",cr2,")(直接比較)",sep=""))}
   if(is.numeric(cr1)){co<- co+1;U[,co]<- ifelse(G[,1]> cr1,    1, 0);
         lab<-c(lab,paste("μ1-μ2が(",cr1,")より大きい確率",sep=""))}
   if(is.numeric(cr3)){co<- co+1;U[,co]<- ifelse(G[,2]> cr3,    1, 0);
         lab<-c(lab,paste("効果量1が(",cr3,")より大きい確率",sep=""));
                       co<- co+1;U[,co]<- ifelse(G[,3]> cr3,    1, 0);
         lab<-c(lab,paste("効果量2が(",cr3,")より大きい確率",sep=""))}
   if(is.numeric(pr1)){co<- co+1;U[,co]<- ifelse(G[,4]> pr1,    1, 0);
         lab<-c(lab,paste("第1群の非重複度が(",pr1,")より大きい確率",sep=""));
                       co<- co+1;U[,co]<- ifelse(G[,5]> pr1,    1, 0);
         lab<-c(lab,paste("第2群の非重複度が(",pr1,")より大きい確率",sep=""))}
   if(is.numeric(pr2)){co<- co+1;U[,co]<- ifelse(G[,6]> pr2,    1, 0);
         lab<-c(lab,paste("優越率が(",pr2,")より大きい確率",sep=""))}
   if((is.numeric(cr2))&(is.numeric(pr3)))
                      {co<- co+1;U[,co]<- ifelse(G[,7]> pr3,    1, 0);
         lab<-c(lab,paste("閾上率(",cr2,")が(",pr3,")より大きい確率",sep=""))}
   if(co>0.9){
      Uc<-matrix(colMeans(as.matrix(U[,1:co])),co,1);
      rownames(Uc)<-lab
      }else{Uc<-0.0}
   waic<- (-2)*(log(mean(exp(log_lik)))) + 2*(var(log_lik))
   print(x$fit,pars=x$par,digits_summary=degits,probs=x$prob)
   print(round(Gc,degits))
   print(round(Uc,degits))
   print(paste("waic=",round(waic,degits),sep=""))
   out<-list(G=G,Gc=Gc,Uc=Uc,waic=waic)
   return(invisible(out))
}  


In [ ]:
import pandas as pd
import requests, zipfile
from io import StringIO
import io
tokyo = requests.get("https://stopcovid19.metro.tokyo.lg.jp/data/130001_tokyo_covid19_patients_2020.csv", stream=True)
tokyoc= pd.read_csv(io.BytesIO(tokyo.content),sep=",")
tokyoc.to_csv("130001_tokyo_covid19_patients_2020.csv", index=False)
#
tokyo = requests.get("https://stopcovid19.metro.tokyo.lg.jp/data/130001_tokyo_covid19_patients_2021.csv", stream=True)
tokyoc= pd.read_csv(io.BytesIO(tokyo.content),sep=",")
tokyoc.to_csv("130001_tokyo_covid19_patients_2021.csv", index=False)
#
tokyo = requests.get("https://stopcovid19.metro.tokyo.lg.jp/data/130001_tokyo_covid19_patients_2022.csv", stream=True)
tokyoc= pd.read_csv(io.BytesIO(tokyo.content),sep=",")
tokyoc.to_csv("130001_tokyo_covid19_patients_2022.csv", index=False)
#
tokyo = requests.get("https://stopcovid19.metro.tokyo.lg.jp/data/130001_tokyo_covid19_patients_2022-1.csv", stream=True)
tokyoc= pd.read_csv(io.BytesIO(tokyo.content),sep=",")
tokyoc.to_csv("130001_tokyo_covid19_patients_2022-1.csv", index=False)
#



In [5]:
tokyo_file_name = "130001_tokyo_covid19_patients_2020.csv"
ari_nashi_tomo_2020 <- 
 read.csv(file = tokyo_file_name, skip = 1, header=F,
           fileEncoding = "UTF-8-BOM"  ) %>%
  select( c( 1, 5, 6, 7, 8, 9, 10, 11, 15, 17 ))  %>%
  rename( no= V1, kohyo= V5, hassho= V6, kakutei= V7, kyojuchi = V8, age = V9, sex = V10, oc = V11, kansen = V15, taiin = V17 ) 

tokyo_file_name = "130001_tokyo_covid19_patients_2021.csv"

ari_nashi_tomo_2021 <- 
 read.csv(file = tokyo_file_name, skip = 1, header=F,
           fileEncoding = "UTF-8-BOM"  ) %>%
  select( c( 1, 5, 6, 7, 8, 9, 10, 11, 15, 17 ))  %>%
  rename( no= V1, kohyo= V5, hassho= V6, kakutei= V7, kyojuchi = V8, age = V9, sex = V10, oc = V11, kansen = V15, taiin = V17 ) 

tokyo_file_name = "130001_tokyo_covid19_patients_2022.csv"

ari_nashi_tomo_2022 <- 
 read.csv(file = tokyo_file_name, skip = 1, header=F,
           fileEncoding = "UTF-8-BOM"  ) %>%
  select( c( 1, 5, 6, 7, 8, 9, 10, 11, 15, 17 ))  %>%
  rename( no= V1, kohyo= V5, hassho= V6, kakutei= V7, kyojuchi = V8, age = V9, sex = V10, oc = V11, kansen = V15, taiin = V17 ) 

tokyo_file_name = "130001_tokyo_covid19_patients_2022-1.csv"

ari_nashi_tomo_2022_1 <- 
 read.csv(file = tokyo_file_name, skip = 1, header=F,
           fileEncoding = "UTF-8-BOM"  ) %>%
  select( c( 1, 5, 6, 7, 8, 9, 10, 11, 15, 17 ))  %>%
  rename( no= V1, kohyo= V5, hassho= V6, kakutei= V7, kyojuchi = V8, age = V9, sex = V10, oc = V11, kansen = V15, taiin = V17 ) 

tokyo_case <- rbind(
	ari_nashi_tomo_2020,
	ari_nashi_tomo_2021,
	ari_nashi_tomo_2022,
  ari_nashi_tomo_2022_1)

write.csv(tokyo_case,"tokyo.csv", row.names = FALSE )

In [6]:
tokyo <- read_csv("tokyo.csv",show_col_types = FALSE) 
df <- tokyo %>%
  select(kohyo, hassho,kakutei) %>%
  filter( !is.na(kakutei)) %>%
  mutate( kohyo = ymd(kohyo)) %>%
  mutate( kakutei = ymd(kakutei )) %>%
  mutate( days = kohyo - kakutei ) %>%
  mutate( hassho_ari = ifelse( !is.na(hassho),2, 1)) %>%
  mutate( kohyo = kohyo - ymd("2020/10/1") + 1 ) %>%
  mutate( kohyo = as.numeric(kohyo )) %>%
  mutate( days = as.numeric(days) ) %>%
  select( kohyo, hassho_ari, days ) 
df<-df[order( df$hassho_ari, df$kohyo ),]
write.csv(df,'df.csv')

In [ ]:
import gc
gc.collect()

210

In [21]:
df <- read.csv('df.csv') %>%
  filter( !is.na(df$days))
# days1[n]~N(mu1[n],sigma1[n])
# days2[n]~N(mu2[n],sigma2[n])
# mu1[n]-mu2[n]
# sigma1[n]-sigma2[n]

#表10.1「知覚時間」データ入力
y1<- df$days
A <- df$kohyo
B <- df$hassho_ari

#2要因実験の推測 例１
out1<-E2Ind(y1,B,prior=T,mL=-100, mH=5000, sL=0, sH=5000, fi=NA,cha=1)
print(out1,3)
#E2betw_level(out1,degits=3,H="B",F=1,I=4,J=3,cr1=1)

no parameter eta2T; sampling not done



Stan model 'E2IndPT_c' does not contain samples.
[1] "******************事前分布指定******************"


ERROR: ignored

In [ ]:
df <- read.csv('df.csv') %>%
  filter( !is.na(df$days))
# days1[n]~N(mu1[n],sigma1[n])
# days2[n]~N(mu2[n],sigma2[n])
# mu1[n]-mu2[n]
# sigma1[n]-sigma2[n]

#表10.1「知覚時間」データ入力
y1<- df$days
A <- df$kohyo
B <- df$hassho_ari

#表5.1の聴音条件のデータ入力
df1 <- df %>%
  filter( hassho_ari == 2)

df2 <- df %>%
  filter( hassho_ari == 1)

x1<- df1$days
#表1.1の対照条件のデータ入力
x2<- df2$days

#表5.2 「知覚データ」の数値要約　
(n1<-length(x1));(n2<-length(x2))         #データの数
mean(x1);mean(x2)                         #平均値
van<-function(x){mean((x-mean(x))^2)}     #分散を計算する関数
van(x1);van(x2)                           #分散
sqrt(van(x1));sqrt(van(x2))               #標準偏差
sort(x1);sort(x2)                         #小さい順に並べる
median(x1);median(x2)                     #中央値
quantile(x1,type =2);quantile(x2,type =2) #％点

#独立した2群の差の推測（標準偏差が異なる）
outDEF<-G2Ind(x1,x2,EQU=0,prior=F)
#MCMC標本の名前が長いので取り出す。
Dmu1<-outDEF$mu1;Dmu2<-outDEF$mu2;Dsigma1<-outDEF$sigma1;
Dsigma2<-outDEF$sigma2;Dxaste1<-outDEF$xaste1;Dxaste2<-outDEF$xaste2;
Dlog_lik<-outDEF$log_lik;


[1] 1275521

[1] 252131

[1] 1.774073

[1] 1.980585

[1] 4.619199

...

[1] -2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   [25]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   [49]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   [73]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   [97]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [121]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [145]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [169]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [193]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [217]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [241]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [265]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [289]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [313]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [337]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [361]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [385]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [409]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [433]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [457]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [481]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [505]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [529]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [553]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [577]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [601]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [625]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [649]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [673]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [697]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [721]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [745]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [769]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [793]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [817]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [841]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [865]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [889]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [913]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [937]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [961]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  [985]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 [1009]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 [1033]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 [1057]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 [1081]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 [1105]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 [1129]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 [1153]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 [1177]  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 [12

[1] 1

[1] 1

0%  25%  50%  75% 100% 
  -1    1    1    2  263

0%  25%  50%  75% 100% 
  -2    1    1    2  369